In [1]:
import os
from google.colab import files
import pandas as pd

#uploaded = files.upload()

#import io

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cd .. && git clone https://github.com/pytorch/fairseq && cd fairseq && git checkout a06083f && pip install --editable .

Cloning into 'fairseq'...
remote: Enumerating objects: 27847, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 27847 (delta 146), reused 195 (delta 123), pack-reused 27579
Receiving objects: 100% (27847/27847), 11.65 MiB | 23.95 MiB/s, done.
Resolving deltas: 100% (20965/20965), done.
Note: checking out 'a06083f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a06083f8 Fix score reference when using translation_from_pretrained_bart task
Obtaining file:///fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  I

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.CC25.tar.gz 
# !tar -xzvf mbart.CC25.tar.gz

In [ ]:
#!wget https://dl.fbaipublicfiles.com/laser/WikiMatrix/v1/WikiMatrix.es-lt.tsv.gz 

In [ ]:
# #@title Default title text
# from tqdm import tqdm, trange
# from sklearn.model_selection import train_test_split

# es_data = []
# lt_data = []

# with open("/content/drive/MyDrive/vgtu/data/WikiMatrix.es-lt.tsv") as fp:
#     for line in tqdm(fp, total=80000):
#         line_data = line.rstrip().split('\t')
#         es_data.append(line_data[1] + '\n')
#         lt_data.append(line_data[2] + '\n')

# total_test = 7000
# es_train, es_subtotal, lt_train, lt_subtotal = train_test_split(
#         es_data, lt_data, test_size=total_test, random_state=42)

# es_test, es_val, lt_test, lt_val = train_test_split(
#         es_subtotal, lt_subtotal, test_size=0.5, random_state=42)

# file_mapping = {
#     'train.es_XX': es_train,
#     'train.lt_XX': lt_train,
#     'valid.es_XX': es_val,
#     'valid.lt_XX': lt_val,
#     'test.es_XX': es_test,
#    'test.lt_XX': lt_test,

# }


# for k, v in file_mapping.items():
#     with open(f'/content/drive/MyDrive/vgtu/data/preprocessed/{k}', 'w+') as fp:
#         fp.writelines(v)


In [3]:
!pip install sentencepiece
import os
import sentencepiece as spm


     |████████████████████████████████| 1.2MB 20.7MB/s 


In [22]:
sp = spm.SentencePieceProcessor()
sp.load("/content/drive/MyDrive/mbart.cc25.v2/sentence.bpe.model")


True

In [32]:
#SPM=/usr/local/bin/spm_encode
#MODEL=/content/mbart.cc25/sentence.bpe.model
DATA="/content/drive/MyDrive/preprocessed"

train_path = "{}/train.csv".format(DATA)
test_path = "{}/test.csv".format(DATA)
val_path = "{}/val.csv".format(DATA)

#TRAIN="train"
#VALID="valid"
#TEST="test"
#SRC="es_XX"
#TGT="lt_XX"

source_suffix='bpe.en_XX'
target_suffix='bpe.lt_LT'
out_dir="/content/drive/MyDrive//postprocessed/"

In [33]:
train_text_file = os.path.join(out_dir, "train.{}".format(source_suffix))
train_summary_file = os.path.join(out_dir, "train.{}".format(target_suffix))
val_text_file = os.path.join(out_dir, "val.{}".format(source_suffix))
val_summary_file = os.path.join(out_dir, "val.{}".format(target_suffix))
test_text_file = os.path.join(out_dir, "test.{}".format(source_suffix))
test_summary_file = os.path.join(out_dir, "test.{}".format(target_suffix))

In [34]:
lowercase=False
max_text_subwords=600
max_summary_subwords=160
insert_tags=False


In [35]:
import pandas as pd

files = ((train_path, train_text_file, train_summary_file),
         (val_path, val_text_file, val_summary_file),
         (test_path, test_text_file, test_summary_file))


i=0
for path, text_file_name, summary_file_name in files:
    with open(text_file_name, "w") as text_file, open(summary_file_name, "w") as summary_file:
        d = pd.read_csv(path, sep='\t') 
        df = zip(d['text'].to_list(), d['summary'].to_list())
        for text, summary in df:
            if lowercase:
                text = text.lower()
                summary = summary.lower()
            text_subwords = sp.EncodeAsPieces(text)
            if max_text_subwords:
                text_subwords = text_subwords[:max_text_subwords]
                if i == 1:
                  print("#### TEXT ####")
                  print(text_subwords)
                  if len(text_subwords) > 599:
                    print(i)
                    print(len(text_subwords))
                    print(text_subwords)
            summary_subwords = sp.EncodeAsPieces(summary)
            if max_summary_subwords:
                summary_subwords = summary_subwords[:max_summary_subwords]
                if i == 1:
                  print("####### summary ############")
                  print(summary_subwords)
            if insert_tags:
                text_subwords.insert(0, "<t>")
                text_subwords.append("</t>")
                summary_subwords.insert(0, "<t>")
                summary_subwords.append("</t>")
            text_file.write(" ".join(text_subwords) + "\n")
            summary_file.write((" ".join(summary_subwords)) + "\n")
            i+=1
            

#### TEXT ####
['▁Pasak', '▁VM', 'VT', '▁specialist', 'ės', '▁Il', 'ono', 's', '▁Dru', 'ly', 'tės', ',', '▁pagrindinė', '▁priežastis', ',', '▁kodėl', '▁iš', 'meta', 'me', '▁tiek', '▁daug', '▁vartoti', '▁tinka', 'mų', '▁maisto', '▁produktų', ',', '▁–', '▁ne', 'supra', 'n', 'tame', '▁ant', '▁pak', 'uočių', '▁nurody', 'tų', '▁gali', 'o', 'jimo', '▁termin', 'ų', '▁reikšmė', 's', '.', '„', 'Nu', 'stato', 'mos', '▁dviejų', '▁rūši', 'ų', '▁maisto', '▁produktų', '▁gali', 'o', 'jimo', '▁termino', '▁pabaigos', '▁datos', '▁–', '▁„', 'tinka', '▁vartoti', '▁iki', '“', '▁arba', '▁„', 'ger', 'iausias', '▁iki', '“.', '▁Angli', 'ška', 'i', '▁jos', '▁vadina', 'si', '▁„', 'use', '▁by', '“', '▁arba', '▁„', 'best', '▁before', '“', '▁–', '▁tai', '▁svarbu', 's', '▁Europos', '▁Sąjungos', '▁teisės', '▁reik', 'al', 'avimas', '.', '▁Ki', 'to', 'kio', '▁ver', 'timo', '▁ir', '▁interpretacij', 'os', '▁negali', '▁būti', ',', '▁nes', '▁termina', 'i', '▁iš', '▁es', 'mės', '▁skiria', 'si', '“,', '▁–', '▁sako', '▁I', '.

In [36]:
!python /fairseq/preprocess.py --source-lang "en_XX" --target-lang "lt_LT" --trainpref "/content/drive/MyDrive/postprocessed/train.bpe" --validpref "/content/drive/MyDrive/postprocessed/val.bpe" --testpref "/content/drive/MyDrive/postprocessed/test.bpe" --destdir "/content/drive/MyDrive/postprocessed/-bin/" --workers 60 --srcdict "/content/drive/MyDrive/mbart.cc25.v2/dict.txt" --tgtdict "/content/drive/MyDrive/mbart.cc25.v2/dict.txt"

2021-05-15 08:59:15 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bpe=None, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/drive/MyDrive/postprocessed/-bin/', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, quantization_config_path=None, seed=1, source_lang='en_XX', srcdict='/content/drive/MyDrive/mbart.cc25.v2/dict.txt', target_lang='lt_LT', task='translation', tensorboard_logdir='', testpref='/content/drive/MyDrive/postprocessed/test.bpe', tgtdict='/content/drive/MyDrive/mbart.cc25.v2/dict.txt', threshold_loss_scale=None, thresholdsr

# TRAIN

In [37]:
!nvidia-smi

Sat May 15 09:02:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# %%writefile setup.sh

# export CUDA_HOME=/usr/local/cuda-10.1
# git clone https://github.com/NVIDIA/apex
# pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
# !sh setup.sh

In [8]:
# --memory-efficient-fp16 \
MAX_UPDATE=80000
WARMUP_UPDATES=500
LR=3e-05
MAX_TOKENS=603
UPDATE_FREQ=2
LANGS="ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN"
SAVEDIR="/content/drive/MyDrive/checkpoint"
DATADIR = "/content/drive/MyDrive/postprocessed/-bin/"


!python /fairseq/train.py \
  $DATADIR \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang en_XX --target-lang lt_LT \
  --ddp-backend=no_c10d \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr "$LR" --warmup-updates "$WARMUP_UPDATES" \
  --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 --clip-norm 0.1 \
  --max-update "$MAX_UPDATE" \
  --max-tokens "$MAX_TOKENS" \
  --required-batch-size-multiple 1 \
  --update-freq "$UPDATE_FREQ" \
  --save-interval 1 --save-interval-updates 5000 \
  --keep-interval-updates 1 --no-epoch-checkpoints \
  --seed 42 --log-format simple --log-interval 100 \
  --restore-file "/content/drive/MyDrive/mbart.cc25.v2/model.pt" \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $LANGS \
  --skip-invalid-size-inputs-valid-test \
  --truncate-source \
  --find-unused-parameters \
  --no-last-checkpoints \
  --save-dir $SAVEDIR

  ## TRY:

    #    --eval-bleu \
    #  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    #  --eval-bleu-detok moses \
    #  --eval-bleu-remove-bpe \
    #  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \

2021-05-15 13:10:32 | INFO | fairseq_cli.train | Namespace(activation_fn='gelu', adam_betas='(0.9, 0.98)', adam_eps=1e-06, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='mbart_large', attention_dropout=0.1, best_checkpoint_metric='loss', bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_suffix='', clip_norm=0.1, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/content/drive/MyDrive/postprocessed/-bin/', data_buffer_size=0, dataset_impl=None, ddp_backend='no_c10d', decoder_attention_heads=16, decoder_embed_dim=1024, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1024, decoder_layerdrop=0, decoder_layers=12, decoder_layers_to_keep=None, decoder_learned_pos=True, decoder_normalize_before=True, decoder_output_dim=1024, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_po

# PREDICT

In [ ]:

from collections import namedtuple

import torch

from fairseq import checkpoint_utils, options, tasks, utils

Batch = namedtuple('Batch', 'ids src_tokens src_lengths')
Translation = namedtuple('Translation', 'src_str hypos pos_scores alignments')

def make_batches(lines, args, task, max_positions, encode_fn):
    tokens = [
        task.source_dictionary.encode_line(
            encode_fn(src_str), add_if_not_exist=False
        ).long()
        for src_str in lines
    ]
    lengths = torch.LongTensor([t.numel() for t in tokens])
    itr = task.get_batch_iterator(
        dataset=task.build_dataset_for_inference(tokens, lengths),
        max_tokens=args.max_tokens,
        max_sentences=args.max_sentences,
        max_positions=max_positions,
    ).next_epoch_itr(shuffle=False)
    for batch in itr:
        yield Batch(
            ids=batch['id'],
            src_tokens=batch['net_input']['src_tokens'], src_lengths=batch['net_input']['src_lengths'],
        )

class Generator():
    def __init__(self, data_path, checkpoint_path="checkpoint_best.pt"):
        self.parser = options.get_generation_parser(interactive=True)
        self.parser.set_defaults(path=checkpoint_path,
            remove_bpe="sentencepiece", dataset_impl="lazy", num_wokers=5
        )
        self.args = options.parse_args_and_arch(self.parser, 
            input_args=[data_path]
        )

        utils.import_user_module(self.args)

        if self.args.buffer_size < 1:
            self.args.buffer_size = 1
        #if self.args.max_tokens is None and self.args.max_sentences is None:
        #    self.args.max_sentences = 1

       # assert not self.args.sampling or self.args.nbest == self.args.beam, \
       #     '--sampling requires --nbest to be equal to --beam'
       # assert not self.args.max_sentences or self.args.max_sentences <= self.args.buffer_size, \
        #    '--max-sentences/--batch-size cannot be larger than --buffer-size'

        self.use_cuda = torch.cuda.is_available() and not self.args.cpu

        self.task = tasks.setup_task(self.args)

        self.models, self._model_args = checkpoint_utils.load_model_ensemble(
            self.args.path.split(':'),
            arg_overrides=eval(self.args.model_overrides),
            task=self.task,
        )

        self.src_dict = self.task.source_dictionary
        self.tgt_dict = self.task.target_dictionary

        for model in self.models:
            model.make_generation_fast_(
                beamable_mm_beam_size=None if self.args.no_beamable_mm else self.args.beam,
                need_attn=self.args.print_alignment,
            )
            if self.args.fp16:
                model.half()
            if self.use_cuda:
                model.cuda()

        self.generator = self.task.build_generator(self.args)

        if self.args.remove_bpe == 'gpt2':
            from fairseq.gpt2_bpe.gpt2_encoding import get_encoder
            self.decoder = get_encoder(
                'fairseq/gpt2_bpe/encoder.json',
                'fairseq/gpt2_bpe/vocab.bpe',
            )
            self.encode_fn = lambda x: ' '.join(map(str, self.decoder.encode(x)))
        else:
            self.decoder = None
            self.encode_fn = lambda x: x

        self.align_dict = utils.load_align_dict(self.args.replace_unk)

        self.max_positions = utils.resolve_max_positions(
            self.task.max_positions(),
            *[model.max_positions() for model in self.models]
        )

    def generate(self, string):
        start_id = 0
        inputs = [string]
        results = []
        for batch in make_batches(inputs, self.args, self.task, self.max_positions, self.encode_fn):
            src_tokens = batch.src_tokens
            src_lengths = batch.src_lengths
            if self.use_cuda:
                src_tokens = src_tokens.cuda()
                src_lengths = src_lengths.cuda()

            sample = {
                'net_input': {
                    'src_tokens': src_tokens,
                    'src_lengths': src_lengths,
                },
            }
            translations = self.task.inference_step(self.generator, self.models, sample)
            for i, (id, hypos) in enumerate(zip(batch.ids.tolist(), translations)):
                src_tokens_i = utils.strip_pad(src_tokens[i], self.tgt_dict.pad())
                results.append((start_id + id, src_tokens_i, hypos))

        for id, src_tokens, hypos in sorted(results, key=lambda x: x[0]):
            if self.src_dict is not None:
                src_str = self.src_dict.string(src_tokens, self.args.remove_bpe)

            for hypo in hypos[:min(len(hypos), self.args.nbest)]:
                hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
                    hypo_tokens=hypo['tokens'].int().cpu(),
                    src_str=src_str,
                    alignment=hypo['alignment'].int().cpu() if hypo['alignment'] is not None else None,
                    align_dict=self.align_dict,
                    tgt_dict=self.tgt_dict,
                    remove_bpe=self.args.remove_bpe,
                )
                if self.decoder is not None:
                    hypo_str = self.decoder.decode(map(int, hypo_str.strip().split()))

                return hypo_str

if __name__ == '__main__':
    gen = Generator("/content/drive/MyDrive/postprocessed/-bin", "/content/drive/MyDrive/checkpoint/small_train/checkpoint_best.pt")

    #print(gen.generate("Koronavirusai yra virusai, kurie cirkuliuoja tarp gyvūnų, tačiau žinoma, kad kai kurie iš jų sukelia infekcijas žmonėms. Sukėlę infekciją žmonėms, jie toliau gali būti perduoti nuo žmogaus žmogui. Koronavirusų infekcijos šaltinis gali būti daugybė gyvūnų. Pavyzdžiui, Artimųjų Rytų respiracinio sindromo koronaviruso (MERS-CoV) šaltinis buvo kupranugariai, o sunkaus ūmaus respiracinio sindromo (SŪRS) - civetės katės. Šis naujas Kinijoje aptiktas koronavirusas yra genetiškai glaudžiai panašus į SŪRS sukeliantį virusą. SŪRS atsirado 2002 m. pabaigoje Kinijoje ir per aštuonis mėnesius 33 šalys pranešė apie daugiau nei 8 000 SŪRS atvejų. Tuo metu nuo šio viruso mirė kas dešimtas susirgęs asmuo. Šiuo metu yra per mažai duomenų, kad būtų galima vertinti mirštamumą nuo COVID-19, tačiau preliminarūs duomenys rodo, kad jis yra mažesnis, nei nuo SŪRS. Nors gripo ir COVID-19 perdavimo keliai ir simptomai yra panašūs, šias infekcijas sukeliantys virusai yra labai skirtingi. Dar labai anksti daryti išvadas, kaip plinta naujasis koronavirusas, tačiau išankstinė informacija rodo, kad jo plitimo mechanizmas yra labiau panašus į SŪRS ar pandeminio gripo, negu į sezoninio gripo plitimo mechanizmą, nes žmonės niekada prieš tai nebuvo susidūrę šia infekcija. Europos ligų prevencijos ir kontrolės centro (ECDC) duomenimis, Europoje nuo gripo ir jo sukeltų komplikacijų kasmet miršta apie 40 000 žmonių. Dabartiniai tyrimai sieja COVID-19 su tam tikrais šikšnosparnių tipais, tačiau neatmetama galimybė, kad infekcijos šaltinis galėjo būti ir kiti gyvūnai. Nėra įrodymų, kad tokie naminiai gyvūnai kaip šunys ar katės gali tapti naujojo koronaviruso šaltiniu, tačiau kontaktuojant su gyvūnais rekomenduojama laikytis bendrų higienos principų."))

RuntimeError: ignored

In [ ]:
MODEL_DIR = "/content/drive/MyDrive/checkpoint/small_train"
!python /fairseq/fairseq_cli/interactive.py /content/drive/MyDrive/postprocessed/-bin \
    --source-lang en_XX --target-lang lt_LT \
    --input /content/drive/MyDrive/evaluation.txt $MODEL_DIR \
    --path $MODEL_DIR/checkpoint_best.pt \
    --batch-size 1 --beam 5

usage: interactive.py [-h] [--no-progress-bar] [--log-interval N]
                      [--log-format {json,none,simple,tqdm}]
                      [--tensorboard-logdir DIR] [--seed N] [--cpu] [--fp16]
                      [--memory-efficient-fp16] [--fp16-no-flatten-grads]
                      [--fp16-init-scale FP16_INIT_SCALE]
                      [--fp16-scale-window FP16_SCALE_WINDOW]
                      [--fp16-scale-tolerance FP16_SCALE_TOLERANCE]
                      [--min-loss-scale D]
                      [--threshold-loss-scale THRESHOLD_LOSS_SCALE]
                      [--user-dir USER_DIR]
                      [--empty-cache-freq EMPTY_CACHE_FREQ]
                      [--all-gather-list-size ALL_GATHER_LIST_SIZE]
                      [--model-parallel-size N]
                      [--checkpoint-suffix CHECKPOINT_SUFFIX]
                      [--quantization-config-path QUANTIZATION_CONFIG_PATH]
                      [--criterion {sentence_ranking,sentence_pre

In [ ]:

# CHECKPOINT_PATH="/content/drive/MyDrive/checkpoint/checkpoint_last.pt" 
# DATA_BIN_PATH="/content/drive/MyDrive/postprocessed/-bin/"
# #SENTENCE_BPE_MODEL="/content/mbart.cc25/sentence.bpe.model"
# DATADIR = "/content/drive/MyDrive/postprocessed/-bin/"
# #OUTPUT_FILE="/content/drive/MyDrive/vgtu/checkpoint/"

# !python /fairseq/generate.py \
#   $DATADIR \
#   --path "$CHECKPOINT_PATH" \
#   --task translation_from_pretrained_bart \
#   --gen-subset test -t en_XX -s lt_LT \
#   --batch-size 2 \
#   --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN \
#   --bpe "sentencepiece" \
#   --sentencepiece-vocab "/content/mbart.cc25";
#   #--scoring sacrebleu > "/content/drive/MyDrive/vgtu/checkpoint/tst.txt";

In [ ]:
# model_dir="/content/drive/MyDrive/checkpoint/" # fix if you moved the checkpoint
# langs=ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN
# fairseq-generate "/content/drive/MyDrive/postprocessed/-bin/" \
#   --path $model_dir/checkpoint_best.pt \
#   --task translation_from_pretrained_bart \
#   --gen-subset test \
#   -t lt_LT -s en_XX \
#   --bpe 'sentencepiece' --sentencepiece-model "/content/drive/MyDrive/checkpoint/sentence.bpe.model" \
#   --sacrebleu --remove-bpe 'sentencepiece' \
#   --batch-size 32 --langs $langs > lt_LT

In [ ]:
!pip install fairseq

     |████████████████████████████████| 1.7MB 7.7MB/s 
     |████████████████████████████████| 133kB 34.2MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 112kB 33.2MB/s 
     |████████████████████████████████| 645kB 33.6MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp37-none-any.whl size=141231 sha256=840f7ef0376f6c2b180df52026c8c371c0c6bb02f477a73097eed5eb9e06ba2a
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built antlr4-python3-runtime
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from fairseq.models.bart import BARTModel

In [ ]:
# VEIKIANTIS
# https://tmramalho.github.io/science/2020/06/10/fine-tune-neural-translation-models-with-mBART/
#from fairseq.models.bart import BARTModel

BASEDIR = '/content/drive/MyDrive/checkpoint/small_train'
bart = BARTModel.from_pretrained(
        BASEDIR,
        checkpoint_file='checkpoint_best.pt',
        bpe='sentencepiece',
        sentencepiece_model=f'{BASEDIR}/sentence.bpe.model')
bart.eval() #for evaluation
#bart.train() to resume training

#sentence_list = ['旅行に来る外国人はこれからも少ないままになりそうです。このため、日本の経済はとても厳しくなっています。']
#translation = bart.sample(sentence_list, beam=5)
#print(translation)
#breakpoint()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (dropout_module): FairseqDropout()
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (dropout_module): FairseqDropout()
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout_module): FairseqDropout()
          (activation_dropout_module): FairseqDr

In [ ]:
# # mbart treniruotas mazu kiekiu tekstu (~1k)

# a = ["Kaip pranešė Policijos departamentas, parlamentaras į policiją kreipėsi apie devintą ryto ir nurodė, kad grasinantį laišką į darbinį elektroninį paštą jis gavo apie penktą valandą ryto. Sostinės policijos atstovė Julija Samorokovskaja BNS sakė, kad E. Sabutis savo pareiškime nurodė, jog laiške grasinama su juo susidoroti dėl to, kad jis yra Seimo narys. Siuntėjo parlamentaras sakė nepažįstantis. Policija pradėjo ikiteisminį tyrimą dėl grasinimo nužudyti ar sunkiai sutrikdyti sveikatą. Laiške prašoma socialinio būsto Laiške, kurį gavo keli parlamentarai rašoma, kad jei asmuo negaus socialinio būsto, „ateisiu pas jus ir subadysiu kaip šunis“. Pateikiame visą laiško tekstą, kalba netaisyta. „Labas Jai jus man neduosite socelinio būso aš ateisiu pas jus ir ir subadisiu kaip šunis galėsite pamatiti daug negyvų žmonių kūnų. Jai duodate sirijos ir Baltarusijos piliečiams kur gyventi tai atsakisite vietos ir man. Kiek aš dar laiko gyvensiu kaip šuva ant gatvės“."]
# b = ["Dėsnis teigia, kad tam tikromis sąlygomis atsitiktinių veiksnių visuma ilgainiui nulemia rezultatą, nepriklausantį arba mažai priklausantį nuo atsitiktinumo. Tikimybių teorijoje ir matematinėje statistikoje didžiųjų skaičių dėsnis teigia, kad vienodomis sąlygomis daug kartų stebint tą patį atsitiktinį dydį stebėtų reikšmių aritmetinis vidurkis artėja prie tam tikros neatsitiktinės reikšmės, vadinamos stebimojo dydžio (teoriniu) vidurkiu. Kai vienodomis sąlygomis ilgą laiką stebimas tas pats įvykis, kurio pasirodymo tikimybė yra p, didžiųjų skaičių dėsnis teigia, kad to įvykio pasirodymų santykinis dažnis artėja prie p. Pvz., neribotai metant simetrišką monetą santykinis herbo atvirtimų dažnis artėja prie teorinės tokio atvirtimo tikimybės, lygios . Vienas didžiųjų skaičių dėsnio matematinių tikslių formulavimų yra toks: jei X1, X2, …, Xn, …, yra nepriklausomųjų vienodai pasiskirsčiusių atsitiktinių dydžių su vidurkiais EXn = a seka, tai aritmetinis vidurkis (X1 + X2 + ... + Xn)/n artėja prie a, kai n neribotai auga. Pagal artėjimo pobūdį (su tikimybe 1 arba pagal tikimybę) skiriama stiprusis arba silpnasis didžiųjų skaičių dėsnis. Dėsnio apibendrinimai nagrinėja atvejus, kai atsitiktiniai dydžiai nėra vienodai pasiskirstę arba nėra nepriklausomi. Nepriklausomųjų atsitiktinių dydžių sumų ribinę elgseną tiksliau aprašo centrinė ribinė teorema. Atskirą didžiųjų skaičių dėsnio atvejį suformulavo ir 1713 paskelbė J. Bernoullis. 1837 apibendrino ir didžiųjų skaičių dėsnio terminą pirmasis pavartojo S. D. Poissonas. 1867 didžiųjų skaičių dėsnio sąvoką praplėtė P. Čebyšovas."]
# c = ["Iki šiol kelionės traukiniais buvo landa, kuria žmonės bandydavo apeiti judėjimo ribojimus, nes griežta kontrolė čia netaikyta. Tačiau Velykų savaitgalį šansų prasmukti be patikros nebeliks, o kontrolė pradėta jau ketvirtadienį. „Šiandien mes patikrinsime keleivius vykstančius traukiniu, kalbinsime juos apie buvimą Vilniuje, kokiomis aplinkybėmis jie atsirado Vilniuje, kur jie ruošiasi važiuoti, kalbinsime ir informuosime juos apie Vyriausybės nutarimo reikalavimus dėl karantino laikymosi ir informuosime apie atsakomybę už to nutarimo sąlygų nesilaikymo“, – kalbėjo Vilniaus miesto trečiojo policijos komisariato viršininkas Donaldas Dubaka. Ketvirtadienio popietę į Kauną vykstantį traukinį lipo kelios dešimtys keleivių. Dauguma jų Kauno gyventojai, atvykę į Vilnių darbo ar medicininiais reikalais. Tiesa, vienas patikrintas jaunuolis negalėjo įtikinamai paaiškinti, kodėl lankėsi Vilniuje. „Susitikti su giminaičiu atvažiavau, tiesiog pasikalbėti – pareigūnui aiškino jaunuolis ir pridūrė. – Tai čia savaitgalį turėtų būti ribojimai, savaitgalį tik griežtesni ribojimai.“ Tačiau jis buvo informuotas, kad karantinas galioja visą laiką. Policininkas užsirašė vaikino duomenis ir įspėjo, kad dabar bus atliekamas tyrimas, o jam gresia bauda, nes be rimtos priežasties atvyko į sostinę ir pažeidė judėjimo ribojimus."]
# d = ["Koronavirusai yra virusai, kurie cirkuliuoja tarp gyvūnų, tačiau žinoma, kad kai kurie iš jų sukelia infekcijas žmonėms. Sukėlę infekciją žmonėms, jie toliau gali būti perduoti nuo žmogaus žmogui. Koronavirusų infekcijos šaltinis gali būti daugybė gyvūnų. Pavyzdžiui, Artimųjų Rytų respiracinio sindromo koronaviruso (MERS-CoV) šaltinis buvo kupranugariai, o sunkaus ūmaus respiracinio sindromo (SŪRS) - civetės katės. Šis naujas Kinijoje aptiktas koronavirusas yra genetiškai glaudžiai panašus į SŪRS sukeliantį virusą. SŪRS atsirado 2002 m. pabaigoje Kinijoje ir per aštuonis mėnesius 33 šalys pranešė apie daugiau nei 8 000 SŪRS atvejų. Tuo metu nuo šio viruso mirė kas dešimtas susirgęs asmuo. Šiuo metu yra per mažai duomenų, kad būtų galima vertinti mirštamumą nuo COVID-19, tačiau preliminarūs duomenys rodo, kad jis yra mažesnis, nei nuo SŪRS. Nors gripo ir COVID-19 perdavimo keliai ir simptomai yra panašūs, šias infekcijas sukeliantys virusai yra labai skirtingi. Dar labai anksti daryti išvadas, kaip plinta naujasis koronavirusas, tačiau išankstinė informacija rodo, kad jo plitimo mechanizmas yra labiau panašus į SŪRS ar pandeminio gripo, negu į sezoninio gripo plitimo mechanizmą, nes žmonės niekada prieš tai nebuvo susidūrę šia infekcija. Europos ligų prevencijos ir kontrolės centro (ECDC) duomenimis, Europoje nuo gripo ir jo sukeltų komplikacijų kasmet miršta apie 40 000 žmonių. Dabartiniai tyrimai sieja COVID-19 su tam tikrais šikšnosparnių tipais, tačiau neatmetama galimybė, kad infekcijos šaltinis galėjo būti ir kiti gyvūnai. Nėra įrodymų, kad tokie naminiai gyvūnai kaip šunys ar katės gali tapti naujojo koronaviruso šaltiniu, tačiau kontaktuojant su gyvūnais rekomenduojama laikytis bendrų higienos principų. "]
# e = ["Burlenčių sportas (kartais vadinamas tiesiog buriavimu) – ekstremalus vandens sportas ir pramoga, kurios pagrindas yra judėjimas vandens paviršiumi panaudojant specialią įrangą, vadinamą burlente. Burlentė yra 2–4 m ilgio plūdrios medžiagos lenta, sujungta su viena bure per lanksčią jungtį. Burlenčių sportas yra buriavimo ir banglenčių sporto hibridas. Burlentė – supaprastintas burinio laivo modelis, kuris vairuojamas palenkiant burę pirmyn arba atgal. Buriuoti burlentėmis galima tiek lygiame vandenyje, tiek bangų mūšoje. Buriavimas burlentėmis yra labai universalus sportas – vieniems tai smagus poilsis prie vandens, kitiems įtemptas lenktyniavimas, dar kitiems – gyvenimo stilius, kurio esmė – tobulos bangos paieškos. Buriavimui reikia ne tiek fizinės jėgos, kiek vikrumo ir lygsvaros. Buriuoti burlentėmis galima esant vėjo greičiui nuo 3 iki 20 m/s, tačiau geriausia, kai vėjo greitis yra tarp 5 ir 15 m/s."]
# f = ["automobilių sporto lenktynės, organizuojamos Tarptautinės Automobilininkų Federacijos FIA (pranc. Fédération Internationale de l'Automobile) ir laikomos automobilių sporto viršūne. Oficialus pavadinimas – FIA Formulės Vienas Pasaulio Čempionatas (angl. FIA Formula One World Championship). Tai – brangiausia sporto šaka pasaulyje, savo istoriją skaičiuojanti nuo 1950-ųjų. Pirmuoju asmeninės įskaitos čempionu tapo italas Giuseppe Farina, o pirmoji komanda, iškovojusi konstruktorių taurę (įsteigtą 1958 m.) – Vanwall ekipa. Sėkmingiausias F1 pilotas, kuriam priklauso beveik visi rekordai – vokietis Michaelis Schumacheris, tarp ekipų išsiskiria italų Ferrari (tai vienintelė komanda, lenktyniaujanti nuo pat pirmojo 1950-ųjų m. sezono) komanda."]
# g = ["Bet meskim visus išvedžiojimus ir eikim tiesiai prie įspūdingų skaičių. Išbandytas ir užgrūdintas 8 litrų variklis generuoja 1578 arklio galias (1177 kW), kaip galėjome įsitikinti, skriedami „Centodieci“ arba „Super Sport 300+“ modeliais. Tačiau „Bugatti“ ties tuo nesustojo ir perkonstravo šį variklį taip, kad jį būtų galima „maitinti“ lenktyniniais degalais, kurių oktaninis skaičius siekia 110. Šis patobulinimas leido pasiekti stulbinamas 1825 AG (1361 kW), esant 7 tūkst. aps./min. ir 1850 niutonmetrų sukimo momentą, esant 2 tūkst. aps./min. Galios šuolis taip pat buvo pasiektas sukūrus naujus keturgubus turbokompresorius.Taikydami įvairias svorio taupymo priemones, inžinieriai iš Molsheimo sugebėjo sumažinti „Bolide“ svorį iki 1240 kilogramų arba maždaug tiek, kiek sveria įprastas europietiškas kompaktiškas automobilis.Nenuostabu, kad toks galingas ir tiek nedaug sveriantis lenktyninis hiperautomobilis yra tikra raketa. Nuo 0 iki 100 km/val. jis įsibėgi vos per 2,17 sekundes, o 200 km/val. greitį pasiekia per 4,36 sekundes. Bet nepamirškite, kad kol kas „Bugatti“ neatliko šių pagreičio testų realiomis sąlygomis – visa tai tėra simuliacijose gauti rezultatai. Tas pats pasakytina ir apie įsibėgėjimą nuo 0 iki 300 km/val. per 7,37 sekundes bei iki 400 km/val. per 12,08 sekundes.„Bugatti“ netgi pateikė teorinį įsibėgėjimo nuo 0 iki 500 km/val. laiką, kuris siekia 20,16 sekundes. Kitaip tariant, „Bolide“ Niurnburgringo trasą turėtų įveikti per 5 minutes ir 23,1 sekundes. Maksimalus greitis? „Gerokai virš 500 km/val.“, o „Le Mano“ rato įveikimo laikas užfiksuotas ties 3 minutėmis ir 7,1 sekundėmis. Vėlgi visa tai tėra simuliacijų rodikliai.Nevertėtų pamiršti ir dizaino. Tik pažiūrėkite į jį. Jei manėte, kad „Bugatti Vision Gran Turismo“ yra iš kito pasaulio, tai „Bolide“ neabejotinai atskriejęs iš tolimos galaktikos. Iš tikrųjų čia slypi kur kas daugiau dalykų, nei galima įžiūrėti plika akimi. Naujasis automobilis gali pasigirti „transformuojama oda“ (kaip „Bugatti“ tai vadina), kurios tikslas – pagerinti oro patekimą į ant stogo esantį ėmiklį. Važiuojant lėtai, ėmiklio paviršius išlieka lygus, tačiau kiek stipriau spustelėjus akceleratorių, jis išsiplečia.Oficialios automobilio nuotraukos ne visiškai tiksliai atspindi jo prošvaisą, kuri siekia vos 995 mm ir yra maždaug 300 mm mažesnė nei įprastas „Chiron“ aukštis. Dėl X formos priekinių ir galinių žibintų „Bolide“ atrodo kaip ką tik išriedėjęs iš mokslinės fantastikos filmo.Vos per 9 mėnesius sukurtas „Bolide“ kol kas nėra įvertintas pinigine išraiška. Taip yra dėl to, kad „Bugatti“ vis dar svarsto, ar verta pradėti šio „aukščiausios klasės vidaus degimo variklio“ gamybą."]


# lst = [a,b,c,d,e,f, g]
# summ = []
# for i,txt in enumerate(lst):
#   print(i)
#   translation = bart.sample(
#     txt, 
#     beam=5, 
#     no_repeat_ngram_size=2,  
#     early_stopping=True)
#   #translation = bart.sample(txt, beam=5, no_repeat_ngram_size=3, do_sample=True)
#   print(translation)
  
#   summ.append(translation)


0
['Parlamentaras Egidijus Sabutis, kuris grasina nužudyti ar sunkiai sutrikdyti sveikatą, prašo socialinio būsto, pranešė policija.[lt_LT]']
1
['Didžiųjų skaičių dėsnis teigia, kad tam tikromis sąlygomis atsitiktinių veiksnių visuma ilgainiui nulemia rezultatą, nepriklausantį arba mažai priklausančią nuo atsititiktinumo.[lt_LT]']
2
['Vilniaus miesto trečiojo policijos komisariato viršininkas Donaldas Dubaka ketvirtadienio popietę patikrino į Kauną vykstantį traukinį ir įspėjo, kad dėl karantino laikymosi bus griežta kontrolė.[lt_LT]']
3
['Koronavirusai yra virusai, kurie cirkuliuoja tarp gyvūnų, tačiau žinoma, kad kai kurie iš jų sukelia infekcijas žmonėms, jie toliau gali būti perduoti nuo žmogaus žmogui.[lt_LT]']
4
['Burlenčių sportas (kartais vadinamas tiesiog buriavimu) – ekstremalus vandens sportos ir pramoga, kurios pagrindas yra judėjimas vandens paviršiumi panaudojant specialią įrangą, vadinamą burlente. Burlentė yra 2–4 m ilgio plūdrios medžiagos lenta, sujungta su viena bure

In [ ]:
# mbart treniruotas 2 epochas apie 40k tekstu

a = ["Kaip pranešė Policijos departamentas, parlamentaras į policiją kreipėsi apie devintą ryto ir nurodė, kad grasinantį laišką į darbinį elektroninį paštą jis gavo apie penktą valandą ryto. Sostinės policijos atstovė Julija Samorokovskaja BNS sakė, kad E. Sabutis savo pareiškime nurodė, jog laiške grasinama su juo susidoroti dėl to, kad jis yra Seimo narys. Siuntėjo parlamentaras sakė nepažįstantis. Policija pradėjo ikiteisminį tyrimą dėl grasinimo nužudyti ar sunkiai sutrikdyti sveikatą. Laiške prašoma socialinio būsto Laiške, kurį gavo keli parlamentarai rašoma, kad jei asmuo negaus socialinio būsto, „ateisiu pas jus ir subadysiu kaip šunis“. Pateikiame visą laiško tekstą, kalba netaisyta. „Labas Jai jus man neduosite socelinio būso aš ateisiu pas jus ir ir subadisiu kaip šunis galėsite pamatiti daug negyvų žmonių kūnų. Jai duodate sirijos ir Baltarusijos piliečiams kur gyventi tai atsakisite vietos ir man. Kiek aš dar laiko gyvensiu kaip šuva ant gatvės“."]
b = ["Dėsnis teigia, kad tam tikromis sąlygomis atsitiktinių veiksnių visuma ilgainiui nulemia rezultatą, nepriklausantį arba mažai priklausantį nuo atsitiktinumo. Tikimybių teorijoje ir matematinėje statistikoje didžiųjų skaičių dėsnis teigia, kad vienodomis sąlygomis daug kartų stebint tą patį atsitiktinį dydį stebėtų reikšmių aritmetinis vidurkis artėja prie tam tikros neatsitiktinės reikšmės, vadinamos stebimojo dydžio (teoriniu) vidurkiu. Kai vienodomis sąlygomis ilgą laiką stebimas tas pats įvykis, kurio pasirodymo tikimybė yra p, didžiųjų skaičių dėsnis teigia, kad to įvykio pasirodymų santykinis dažnis artėja prie p. Pvz., neribotai metant simetrišką monetą santykinis herbo atvirtimų dažnis artėja prie teorinės tokio atvirtimo tikimybės, lygios . Vienas didžiųjų skaičių dėsnio matematinių tikslių formulavimų yra toks: jei X1, X2, …, Xn, …, yra nepriklausomųjų vienodai pasiskirsčiusių atsitiktinių dydžių su vidurkiais EXn = a seka, tai aritmetinis vidurkis (X1 + X2 + ... + Xn)/n artėja prie a, kai n neribotai auga. Pagal artėjimo pobūdį (su tikimybe 1 arba pagal tikimybę) skiriama stiprusis arba silpnasis didžiųjų skaičių dėsnis. Dėsnio apibendrinimai nagrinėja atvejus, kai atsitiktiniai dydžiai nėra vienodai pasiskirstę arba nėra nepriklausomi. Nepriklausomųjų atsitiktinių dydžių sumų ribinę elgseną tiksliau aprašo centrinė ribinė teorema. Atskirą didžiųjų skaičių dėsnio atvejį suformulavo ir 1713 paskelbė J. Bernoullis. 1837 apibendrino ir didžiųjų skaičių dėsnio terminą pirmasis pavartojo S. D. Poissonas. 1867 didžiųjų skaičių dėsnio sąvoką praplėtė P. Čebyšovas."]
c = ["Iki šiol kelionės traukiniais buvo landa, kuria žmonės bandydavo apeiti judėjimo ribojimus, nes griežta kontrolė čia netaikyta. Tačiau Velykų savaitgalį šansų prasmukti be patikros nebeliks, o kontrolė pradėta jau ketvirtadienį. „Šiandien mes patikrinsime keleivius vykstančius traukiniu, kalbinsime juos apie buvimą Vilniuje, kokiomis aplinkybėmis jie atsirado Vilniuje, kur jie ruošiasi važiuoti, kalbinsime ir informuosime juos apie Vyriausybės nutarimo reikalavimus dėl karantino laikymosi ir informuosime apie atsakomybę už to nutarimo sąlygų nesilaikymo“, – kalbėjo Vilniaus miesto trečiojo policijos komisariato viršininkas Donaldas Dubaka. Ketvirtadienio popietę į Kauną vykstantį traukinį lipo kelios dešimtys keleivių. Dauguma jų Kauno gyventojai, atvykę į Vilnių darbo ar medicininiais reikalais. Tiesa, vienas patikrintas jaunuolis negalėjo įtikinamai paaiškinti, kodėl lankėsi Vilniuje. „Susitikti su giminaičiu atvažiavau, tiesiog pasikalbėti – pareigūnui aiškino jaunuolis ir pridūrė. – Tai čia savaitgalį turėtų būti ribojimai, savaitgalį tik griežtesni ribojimai.“ Tačiau jis buvo informuotas, kad karantinas galioja visą laiką. Policininkas užsirašė vaikino duomenis ir įspėjo, kad dabar bus atliekamas tyrimas, o jam gresia bauda, nes be rimtos priežasties atvyko į sostinę ir pažeidė judėjimo ribojimus."]
d = ["Koronavirusai yra virusai, kurie cirkuliuoja tarp gyvūnų, tačiau žinoma, kad kai kurie iš jų sukelia infekcijas žmonėms. Sukėlę infekciją žmonėms, jie toliau gali būti perduoti nuo žmogaus žmogui. Koronavirusų infekcijos šaltinis gali būti daugybė gyvūnų. Pavyzdžiui, Artimųjų Rytų respiracinio sindromo koronaviruso (MERS-CoV) šaltinis buvo kupranugariai, o sunkaus ūmaus respiracinio sindromo (SŪRS) - civetės katės. Šis naujas Kinijoje aptiktas koronavirusas yra genetiškai glaudžiai panašus į SŪRS sukeliantį virusą. SŪRS atsirado 2002 m. pabaigoje Kinijoje ir per aštuonis mėnesius 33 šalys pranešė apie daugiau nei 8 000 SŪRS atvejų. Tuo metu nuo šio viruso mirė kas dešimtas susirgęs asmuo. Šiuo metu yra per mažai duomenų, kad būtų galima vertinti mirštamumą nuo COVID-19, tačiau preliminarūs duomenys rodo, kad jis yra mažesnis, nei nuo SŪRS. Nors gripo ir COVID-19 perdavimo keliai ir simptomai yra panašūs, šias infekcijas sukeliantys virusai yra labai skirtingi. Dar labai anksti daryti išvadas, kaip plinta naujasis koronavirusas, tačiau išankstinė informacija rodo, kad jo plitimo mechanizmas yra labiau panašus į SŪRS ar pandeminio gripo, negu į sezoninio gripo plitimo mechanizmą, nes žmonės niekada prieš tai nebuvo susidūrę šia infekcija. Europos ligų prevencijos ir kontrolės centro (ECDC) duomenimis, Europoje nuo gripo ir jo sukeltų komplikacijų kasmet miršta apie 40 000 žmonių. Dabartiniai tyrimai sieja COVID-19 su tam tikrais šikšnosparnių tipais, tačiau neatmetama galimybė, kad infekcijos šaltinis galėjo būti ir kiti gyvūnai. Nėra įrodymų, kad tokie naminiai gyvūnai kaip šunys ar katės gali tapti naujojo koronaviruso šaltiniu, tačiau kontaktuojant su gyvūnais rekomenduojama laikytis bendrų higienos principų. "]
e = ["Burlenčių sportas (kartais vadinamas tiesiog buriavimu) – ekstremalus vandens sportas ir pramoga, kurios pagrindas yra judėjimas vandens paviršiumi panaudojant specialią įrangą, vadinamą burlente. Burlentė yra 2–4 m ilgio plūdrios medžiagos lenta, sujungta su viena bure per lanksčią jungtį. Burlenčių sportas yra buriavimo ir banglenčių sporto hibridas. Burlentė – supaprastintas burinio laivo modelis, kuris vairuojamas palenkiant burę pirmyn arba atgal. Buriuoti burlentėmis galima tiek lygiame vandenyje, tiek bangų mūšoje. Buriavimas burlentėmis yra labai universalus sportas – vieniems tai smagus poilsis prie vandens, kitiems įtemptas lenktyniavimas, dar kitiems – gyvenimo stilius, kurio esmė – tobulos bangos paieškos. Buriavimui reikia ne tiek fizinės jėgos, kiek vikrumo ir lygsvaros. Buriuoti burlentėmis galima esant vėjo greičiui nuo 3 iki 20 m/s, tačiau geriausia, kai vėjo greitis yra tarp 5 ir 15 m/s."]
f = ["automobilių sporto lenktynės, organizuojamos Tarptautinės Automobilininkų Federacijos FIA (pranc. Fédération Internationale de l'Automobile) ir laikomos automobilių sporto viršūne. Oficialus pavadinimas – FIA Formulės Vienas Pasaulio Čempionatas (angl. FIA Formula One World Championship). Tai – brangiausia sporto šaka pasaulyje, savo istoriją skaičiuojanti nuo 1950-ųjų. Pirmuoju asmeninės įskaitos čempionu tapo italas Giuseppe Farina, o pirmoji komanda, iškovojusi konstruktorių taurę (įsteigtą 1958 m.) – Vanwall ekipa. Sėkmingiausias F1 pilotas, kuriam priklauso beveik visi rekordai – vokietis Michaelis Schumacheris, tarp ekipų išsiskiria italų Ferrari (tai vienintelė komanda, lenktyniaujanti nuo pat pirmojo 1950-ųjų m. sezono) komanda."]
g = ["Bet meskim visus išvedžiojimus ir eikim tiesiai prie įspūdingų skaičių. Išbandytas ir užgrūdintas 8 litrų variklis generuoja 1578 arklio galias (1177 kW), kaip galėjome įsitikinti, skriedami „Centodieci“ arba „Super Sport 300+“ modeliais. Tačiau „Bugatti“ ties tuo nesustojo ir perkonstravo šį variklį taip, kad jį būtų galima „maitinti“ lenktyniniais degalais, kurių oktaninis skaičius siekia 110. Šis patobulinimas leido pasiekti stulbinamas 1825 AG (1361 kW), esant 7 tūkst. aps./min. ir 1850 niutonmetrų sukimo momentą, esant 2 tūkst. aps./min. Galios šuolis taip pat buvo pasiektas sukūrus naujus keturgubus turbokompresorius.Taikydami įvairias svorio taupymo priemones, inžinieriai iš Molsheimo sugebėjo sumažinti „Bolide“ svorį iki 1240 kilogramų arba maždaug tiek, kiek sveria įprastas europietiškas kompaktiškas automobilis.Nenuostabu, kad toks galingas ir tiek nedaug sveriantis lenktyninis hiperautomobilis yra tikra raketa. Nuo 0 iki 100 km/val. jis įsibėgi vos per 2,17 sekundes, o 200 km/val. greitį pasiekia per 4,36 sekundes. Bet nepamirškite, kad kol kas „Bugatti“ neatliko šių pagreičio testų realiomis sąlygomis – visa tai tėra simuliacijose gauti rezultatai. Tas pats pasakytina ir apie įsibėgėjimą nuo 0 iki 300 km/val. per 7,37 sekundes bei iki 400 km/val. per 12,08 sekundes.„Bugatti“ netgi pateikė teorinį įsibėgėjimo nuo 0 iki 500 km/val. laiką, kuris siekia 20,16 sekundes. Kitaip tariant, „Bolide“ Niurnburgringo trasą turėtų įveikti per 5 minutes ir 23,1 sekundes. Maksimalus greitis? „Gerokai virš 500 km/val.“, o „Le Mano“ rato įveikimo laikas užfiksuotas ties 3 minutėmis ir 7,1 sekundėmis. Vėlgi visa tai tėra simuliacijų rodikliai.Nevertėtų pamiršti ir dizaino. Tik pažiūrėkite į jį. Jei manėte, kad „Bugatti Vision Gran Turismo“ yra iš kito pasaulio, tai „Bolide“ neabejotinai atskriejęs iš tolimos galaktikos. Iš tikrųjų čia slypi kur kas daugiau dalykų, nei galima įžiūrėti plika akimi. Naujasis automobilis gali pasigirti „transformuojama oda“ (kaip „Bugatti“ tai vadina), kurios tikslas – pagerinti oro patekimą į ant stogo esantį ėmiklį. Važiuojant lėtai, ėmiklio paviršius išlieka lygus, tačiau kiek stipriau spustelėjus akceleratorių, jis išsiplečia.Oficialios automobilio nuotraukos ne visiškai tiksliai atspindi jo prošvaisą, kuri siekia vos 995 mm ir yra maždaug 300 mm mažesnė nei įprastas „Chiron“ aukštis. Dėl X formos priekinių ir galinių žibintų „Bolide“ atrodo kaip ką tik išriedėjęs iš mokslinės fantastikos filmo.Vos per 9 mėnesius sukurtas „Bolide“ kol kas nėra įvertintas pinigine išraiška. Taip yra dėl to, kad „Bugatti“ vis dar svarsto, ar verta pradėti šio „aukščiausios klasės vidaus degimo variklio“ gamybą."]


lst = [a,b,c,d,e,f, g]
summ = []
for i,txt in enumerate(lst):
  print(i)
  translation = bart.sample(txt, beam=4, lenpen=2.0, max_len_b=140, min_len=55, no_repeat_ngram_size=3)
  print(translation)
  
  summ.append(translation)


0
['Lietuvos socialdemokratų partijos (LSDP) frakcijos narys Eugenijus Sabutis pirmadienį gavo laišką, kuriame grasinama nužudyti ir sunkiai sutrikdyti sveikatą, o jei asmuo negaus socialinio būsto, jis bus subadytas kaip šunis.[lt_LT]']
1
['Didžiųjų skaičių dėsnis teigia, kad tam tikromis sąlygomis atsitiktiniai dydžiai nėra vienodai pasiskirstę arba nėra nepriklausomi nuo atsitiktinumo, rašoma pranešime žiniasklaidai. Dėsnio apibendrinimai nagrinėja atvejus, kai vienodomis sąlygomis ilgą laiką stebimas tas pats įvykis, kurio pasirodymo tikimybė yra p.[lt_LT]']
2
['į traukinį lipo kelios dešimtys keleivių. Dauguma jų atvykę į Vilnių darbo ar medicininiais reikalais, o vienas į Kauną atvykęs jaunuolis negalėjo įtikinamai paaiškinti, kodėl atvyko į sostinę ir pažeidė karantino reikalavimus.[lt_LT]']
3
['koronavirusas yra genetiškai glaudžiai panašus į gripo sukeliantį virusą, tačiau žmonės niekada prieš tai nebuvo susidūrę šia infekcija, rašoma pranešime spaudai, remdamasis Europos ligų

In [ ]:
test_article = ["Susitelkti teko visiems Praeitos savaitės pabaigoje progimnazijos Tėvų tarybos pirmininkas Arūnas Penkaitis sakė, kad grįžimo į mokyklą laukiama kaip šventės, kaip gimtadienio: „Anksčiau, kai išties turėjome tam tikrą informacijos trūkumą, nuotaikų buvo įvairių. Šiuo metu, gavus atsakymus į visus kilusius klausimus, didžioji dauguma bendruomenės nusiteikusi labai pozityviai.“ Mokyklos direktorius R.Remeika praėjusią savaitę 15min teigė, kad nuotolinis ugdymas nėra toks efektyvus, todėl siekiama kuo greičiau pereiti į kontaktinį mokymą. Pirmadienio rytą jis feisbuke džiaugėsi, kad bendruomenė susitelkė ir atsakingai dirbdama parodė, kiek daug vaikų labui gali padaryti. O ši savaitė taip pat būsianti susitelkimo metas, kai teks griežtai laikytis visų saugumo taisyklių tiek mokykloje, tiek namuose, tiek viešoje erdvėje, nes esą nėra „prabangos“ susirgti ir taip rizikuoti aplinkinių sveikata. Rezultatus vertins po dar vienų testų Anot Vilniaus mero Remigijaus Šimašiaus, prieš grįžtant į klases buvo testuojami ir darbuotojai, ir mokiniai, ir jų šeimų nariai, tad iššūkių netrūko. „Savaitės pabaigoje vėl bus atliekami testai, žiūrėsime, kokie rezultatai. Nes patys suprantame, kad švietimas yra ta sritis, kur labai norisi grįžti į normalias vėžes ir dar geresnes – per pandemiją išmoko naujų dalykų ir mokytojai, ir mokiniai. Kartu suprantame tą riziką – iš Estijos gavome ataskaitą, kad mokyklose didelė rizika, vaikai vieni kitiems perneša. Mūsų tą eksperimentą Lietuvos mastu traktuoju kaip labai svarbų – kaip organizuoti ir kaip paskui viskas vyks“, – kalbėjo R.Šimašius. Mero teigimu, jeigu reikėtų pagal tokią pačią procedūrą atidaryti visas Lietuvos ar Vilniaus mokyklas, tai greičiausia nebūtų įmanoma – resursų niekam ir niekada neužtektų. Todėl reikia apibendrinti išvadas, kaip tai galima padaryti paprasčiau. Apie tai ketinama diskutuoti ir su Vyriausybe. Atsidarytų nors rytoj Paklaustas, kada galėtų visos mokyklos atsidaryti, meras sakė, kad jeigu virusas iš Lietuvos išeitų, tai galima būtų ir rytoj padaryti. „Bet kada jis išeis, kokiu tempu, kokios variacijos, kiti dalykai, negaliu pasakyti. Bet jeigu epidemiologinė situacija leistų, sugrįžti į mokyklas mums nebūtų problemų. Nebent kiltų vienai kitai šeimai, nes ne paslaptis, kad ne vienas moksleivis, kartais ir mokytojai pamokose dalyvauja ne iš Vilniaus, kartais net ir į užsienį išvažiavę“, – sakė R.Šimašius. Jis įsitikinęs, kad tokių ar panašių eksperimentų turėtų būti ir daugiau, tai leistų lengviau grįžti visiems, kai epidemiologinė situacija leis. Vilnius rengė konferenciją su mokytojais, svarstė nuotolinio mokymosi sąlygas ir būdus, tai joje paaiškėjo, kad tiek abiturientai, tiek mokytojai vieningai sako, kad egzaminų vėlinti negalima ir nereikia, yra pasiūlymų, kaip organizuoti juos tinkamai įprastu metu. Buvo teigiamų testų Vilniaus visuomenės sveikatos biuro vadovė Guoda Ropaitė-Beigė teigė, kad viso šio proceso metu mokyklos bendruomenė buvo itin aktyvi: „Ketvirtadienį startavome su pedagogų testavimu kaupinių principu, tai 93 asmenys, sudarėme 39 kaupinius, visi buvo neigiami. Tai tikrai mokytojai saugūs ir sveiki grįžo į šitą įstaigą. O kalbant apie namų ūkius – t. y. vaikus ir jų tėvelius, testavimas vyko penktadienį, šeštadienį ir dar sudarėme galimybę ir sekmadienį testuotis. Buvo per 500 kaupinių, daugiau nei 2 tūkst. asmenų atvyko testuotis į testavimo punktą, keturi kaupiniai buvo teigiami, dabar vykdomi tyrimai PGR tyrimai.“ Šis testavimo būdas skiriasi nuo PGR tuo, kad pastarojo atveju imamas mėginys iš nosiaryklės ir ryklės, o kaupinio – iš nosies landų ir tie ėminiai grupuojami: vienoje virusologinėje terpėje grupuojami ir dedami kelių asmenų ėminius ir jie tiriami."]

translation = bart.sample(test_article, beam=4, lenpen=2.0, max_len_b=140, min_len=55, no_repeat_ngram_size=3)
print(translation)


['Pirmadienio popietę Vilniaus progimnazijos direktorius Remigijus Remeika feisbuke džiaugėsi, kad bendruomenė susitelkė ir atsakingai dirbdama parodė, kiek daug vaikų labui gali padaryti, o ši savaitė taip pat busianti susitelkimo metas, kai teks griežtai laikytis visų saugumo taisyklių tiek mokykloje, tiek namuose, tiek viešoje erdvėje, nes esą nėra „prabangos“.[lt_LT]']


In [ ]:
test_article = ["""
Oficialiosios statistikos įstaigos, įvairios valstybės tarnybos, medicinos įstaigos, mobiliojo ryšio operatoriai, bankai, transporto ir kitų verslo sričių įmonės, socialiniai tinklai, žaidimų portalai ir didžiulė įvairovė kitų internete veikiančių ir paslaugas teikiančių svetainių sukaupia milžiniškus kiekius duomenų.
Be to, atsiranda vis daugiau  išmaniųjų įrenginių, kurie taip pat generuoja didžiulius kiekius įvairios prigimties duomenų. Jų analizė tampa didele problema. Norint iš turimų duomenų gauti prasmingos ir naudingos informacijos verslui, reikia naujos kartos analitikų, gebančių taikyti šiuolaikinius analizės ir prognozavimo metodus, technologijas ir analitikos programines priemones.
Tai – taikomosios statistikos specialisto darbas, kuris šiuolaikiniame pasaulyje vis reikalingesni.
Į Taikomosios statistikos magistro studijų programą gali stoti statistikos, matematikos arba informatikos bakalauro kvalifikacinį laipsnį turintys studentai.
Baigus studijas suteikiamas matematikos mokslų magistro laipsnis.
Taikomosios statistikos magistro studijų programoje galima rinktis vieną iš dviejų specializacijų: Duomenų mokslą arba Statistinius metodus finansuose ir ekonomikoje.
Duomenų mokslo specializacija orientuota į duomenų analizės specialistų rengimą.
Analizuojant ypač didelius duomenų kiekius susiduriama su specifinėmis problemomis, kokių tradicinėje statistikoje iš esmės nėra. Pirmiausia, tai įvairios problemos, kylančios dėl fizinės duomenų apimties. Dėl tos pačios priežasties smarkiai išauga duomenų apdorojimo laikas.
Kita aktuali problema yra ta, kad labai dažnai duomenys yra nestruktūrizuoti, todėl tradicinių statistinės analizės metodų tiesiogiai pritaikyti negalima. Be to, atsiranda tokių uždavinių, kuriems reikia naujų duomenų analizės metodų.
Tai paskatino naujos tarpdisciplininės mokslo srities – duomenų mokslo (angl. Data Science) – vystymąsi.
Kvalifikuotai didelės apimties duomenų analizei reikalingi tokie specialistai, kurie mokėtų programuoti, turėtų žinių apie duomenų bazes ir kitas su duomenų apdorojimu susijusias informacines technologijas. Be to, žinotų ir mokėtų taikyti statistinės analizės, statistinio modeliavimo bei prognozavimo metodus. Tokių specialistų poreikis tik didėja.
Statistinių metodų finansuose ir ekonomikoje specializacija yra skirta rengti šiuolaikinius duomenų analizėje taikomus modelius ir metodus išmanančius bei būtiniausių darbo su realia statistine informacija įgūdžių turinčius specialistus.
Programa atitinka taikomosios ekonometrijos pakraipą ir yra aktuali, kadangi  Lietuvoje itin stinga šios srities specialistų. Šiuolaikiniame versle statistinės analizės svarba nuolat auga, o reikiamos kvalifikacijos profesionalų Lietuvoje yra nedaug.
Programos tikslas – rengti aukštos kvalifikacijos specialistus, gebančius matematinės statistikos metodais atlikti svarbių ir aktualių Lietuvai ekonominio ir techninio pobūdžio duomenų analizę, prekių ir paslaugų kainų dinamikos analizę ir prognozę, šiuolaikiniais statistikos metodais nustatyti optimalių draudimo įnašų dydžius ir t. t.
"""
]

translation = bart.sample(test_article, beam=4, lenpen=2.0, max_len_b=140, min_len=55, no_repeat_ngram_size=3)
print(translation)

['ikonominės, finansinės, socialinės ir politikos srityse dirbantys specialistai turi žinių apie duomenų bazes ir kitas su duomenų apdorojimu susijusias informacines technologijas. Tai – taikomosios statistikos magistro studijų programos dalis, kurioje galima rinktis vieną iš dviejų specializacijų: Duomenų mokslą arba Statistinius metodus finansuose ir ekonomikoje.[lt_LT]']


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizerfrom nltk import sent_tokenize, word_tokenize